# iPython notebook to recursively apply permissions to Projects
## Importing libraries

In [1]:
import os, argparse, keyring, re, configparser, warnings, urllib, requests, webbrowser
from datetime import time, datetime
from getpass import getpass
from pathlib import Path
warnings.filterwarnings('ignore')
config = configparser.ConfigParser()
config.read(r".\project_permissions.cfg")

['C:\\Users\\pnl0dv6z\\OneDrive - AholdDelhaize.com\\Documents\\Python Scripts\\Project_permissions_applier\\project_permissions.cfg']

## Functions
### Fn to check errors

In [12]:
def check_error(request, task):
    if task == "sign_in":
        if request.status_code == 200:
            print("\t\tUser signed in successfully!")
            return 1
        elif request.status_code == 404:
            print("\t\tERROR: User not found!!")
            return 0
        elif request.status_code == 401:
            print("\t\tERROR: Login error!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
        
    elif task == "sign_out":
        if request.status_code == 204:
            print("\t\tUser signed out successfully!")
            return 1
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
        
    elif task == "create_users":
        if request.status_code == 201:
            print("\t\tUser created successfully!")
            return 1
        elif request.status_code == 404:
            print("\t\tERROR: Site not found!!")
            return 0
        elif request.status_code == 409:
            print("\t\tERROR: User exists or license unavailable, check again!!")
            return 0
        elif request.status_code == 400:
            print("\t\tERROR: Invalid site role or bad request!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
        
    elif task == "update_users":
        if request.status_code == 200:
            print("\t\tUser information updated successfully!")
            return 1
        elif request.status_code == 404:
            print("\t\tERROR: User or Site not found!!")
            return 0
        elif request.status_code == 409:
            print("\t\tERROR: User exists or license unavailable, check again!!")
            return 0
        elif request.status_code == 400:
            print("\t\tERROR: Invalid site role, email address or bad request!!")
            return 0
        elif request.status_code == 403:
            print("\t\tERROR: Licensing update on self or guest account not allowed!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
        
    elif task == "find_group_id":
        if request.status_code == 200:
            print("\t\tGroup found!")
            return 1
        elif request.status_code == 404:
            print("\t\tERROR: Site not found!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
        
    elif task == "add_user_group":
        if request.status_code == 200:
            print("\t\tUser added to group successfully!")
            return 1
        elif request.status_code == 404:
            print("\t\tERROR: Site or Group not found!!")
            return 0
        elif request.status_code == 409:
            print("\t\tERROR: Specified User already in group!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0
    
    elif task == "query_projects":
        if request.status_code == 200:
            print("\t\tProject found!")
            return 1
        elif request.status_code == 400:
            print("\t\tERROR: Pagination error!!")
            return 0
        elif request.status_code == 403:
            print("\t\tERROR: Page not found!!")
            return 0
        elif request.status_code == 404:
            print("\t\tERROR: Site not found!!")
            return 0
        else:
            print("\t\tERROR: Request error check again!!")
            return 0


### Fn to sign in to Server

In [3]:
def sign_in(username, password, site=""):
    body = {
        "credentials": {
            "name": username,
            "password": password,
            "site": {
                "contentUrl": site
            }
        }
    }
    response = requests.post(
        URL + '/auth/signin', 
        json=body, 
        verify=False, 
        headers={'Accept': 'application/json'}
    )
    
    status = check_error(response, "sign_in")
    if status:
        return response.json()['credentials']['site']['id'], response.json()['credentials']['token']
    else:
        return 0,0

### Fn to sign out from Server

In [4]:
def sign_out(site_id, token):
    response = requests.post(
        URL + '/auth/signout', 
        verify=False, 
        headers={'Accept': 'application/json',
                'X-Tableau-Auth': token}
    )
    status = check_error(response, "sign_out")
    return status

### Fn to find project ids

In [73]:
def query_projects(site_id, token, project_names):
    projects = []
    if len(project_names) > 0:
        for project in project_names:
            response = requests.get(
                URL + '/sites/{}/projects?filter=name:eq:{}'.format(site_id, urllib.parse.quote_plus(project)), 
                verify=False, 
                headers={'Accept': 'application/json',
                        'X-Tableau-Auth': token}
            )
            status = check_error(response, "query_projects")
            temp = {response.json()['projects']['project'][0]['name']:response.json()['projects']['project'][0]['id']}
            projects.append(temp)
    else:
        response = requests.get(
            URL + '/sites/{}/projects'.format(site_id), 
            verify=False, 
            headers={'Accept': 'application/json',
                    'X-Tableau-Auth': token}
        )
        status = check_error(response, "query_projects")
        page_size = int(response.json()['pagination']['pageSize'])
        items_available = int(response.json()['pagination']['totalAvailable'])
        if items_available%page_size > 0:
            num_pages = int(items_available/page_size) + 1
        else:
            num_pages = int(items_available/page_size)
        for page_num in range(num_pages):
            response = requests.get(
                URL + '/sites/{}/projects?pageSize={}&pageNumber={}'.format(site_id, page_size, int(page_num)+1), 
                headers={'Accept': 'application/json',
                        'X-Tableau-Auth': token}
            ).json()
            project_names = response['projects']['project']
            for project in project_names:
                temp = {project['name']:project['id']}
                projects.append(temp)
    return projects

### Fn to apply Project specific permissions

In [ ]:
def apply_project_permissions(site_id, token, user, capabilities):
     project_permissions_body = {
        "permissions": {
            "granteeCapabilities": {
                "user" : {
                    "id" : user
                }
                "capabilities" : {
                    
                }
            } user_name,
            "siteRole": site_role
        }
    }

## Variables

In [6]:
server = config["server_connection"]["server"] # Enter site in format tableau.company.com without the https before it
site_content_url = config["server_connection"]["site"] # This can be found from the URL of the content and if using the Default site then this will be blank
api_ver = config["server_connection"]["api"] # This can be found from the Tableau Server REST API reference
username = "pnl0dv6z" # This is your username

URL = "https://{}/api/{}".format(server, api_ver)

## Open Log file

In [7]:
log_file_loc = r"{}\{}".format(str(Path.home()), config["logging_details"]["logfilename"])
log_file = open(log_file_loc, "a+")

## Sign in to the Tableau Server

In [9]:
password = getpass("Enter your password for the Tableau Server: ")
site_id, token = sign_in(username, password, site_content_url)
if token == 0:
    exit()

Enter your password for the Tableau Server:  ············


		User signed in successfully!


In [74]:
test = query_projects(site_id, token, [])

		Project found!


In [76]:
print(len(test))

186


In [59]:
response = requests.get(
    URL + '/sites/{}/projects?pageSize=100&pageNumber=5'.format(site_id), 
    headers={'Accept': 'application/json',
            'X-Tableau-Auth': token}
).json()
print(response)

{'error': {'summary': 'Bad Request', 'detail': "Invalid page number '5'", 'code': '400006'}}


In [63]:
n = 100
d = 100
print(n%d)

0


In [72]:
print(test)

[{'Default': 'b3ff80a2-4208-11e7-aff1-ebade968559c'}, {'Online Lab': 'f91c5633-0d1c-42a2-86b1-9f5c326fb4b6'}, {'Online Live': 'd989f4e3-22b3-4f78-b54e-ffc70138a784'}, {'Marketing & Format Lab': '08588e4d-5c9b-40dd-bb28-f7a60d9f7059'}, {'Marketing & Format Live': 'dc904b82-90bd-4c62-89e3-bdec72abe0b6'}, {'Shared Live': '2cc5cb90-213d-4b13-a38f-faa03a65d428'}, {'Analytics Lab': 'fafdb345-5fcb-4dff-9f7d-a32e0df73c97'}, {'Merchandising & Sourcing Lab': 'f9bd72f3-a739-4a51-8722-0ba52ff5b921'}, {'Administration': '9cbfc4ab-397d-40fa-9998-db338d9893a6'}, {'Formule Lab': 'acdfb2d0-e69d-4e29-9750-1f6dcca3d21d'}, {'M&S Dashboards': '6f7bff85-880b-418a-83da-9e9e3da27a0f'}, {'Finance Stores Lab': '9b6eda6e-a72a-42ea-9b89-601baf980dad'}, {'AHMS Lab': '3c46aae1-9ae0-48fa-94c2-224da74da089'}, {'AH Store Support Lab': '786a0981-151b-42ad-aceb-79247505b98e'}, {'AH Store Support Live': '1c1f51f7-a04c-4701-8484-2447f877141a'}, {'Online Playground (analist-only)': '9756f3a4-b0f6-421c-89b4-b76f250502b8'}, 

In [77]:
urllib.parse.quote_plus('analytics_service')

'analytics_service'

In [1]:
test_dict = {'write':'allow', 'read':'allow'}

In [9]:
cap_dict = []
for i in test_dict:
    temp = {}
    temp['capability'] = {'name': i, 'mode' : test_dict[i]}
    cap_dict.append(temp)
print(cap_dict)

[{'capability': {'name': 'write', 'mode': 'allow'}}, {'capability': {'name': 'read', 'mode': 'allow'}}]


In [20]:
project_permissions_body = {
        "permissions": {
            "granteeCapabilities": {
                "user" : {
                    "id" : 'pnl0dv6z'
                }
                "capability":
                    cap_dict
            }
        }
    }

SyntaxError: invalid syntax (<ipython-input-20-d9496f82b494>, line 7)

In [16]:
type(project_permissions_body)

dict